In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import sys, os, math
import torch
import json

device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
print(device)

epochs = 100_000
val_epoch = 1000
num_val = 100
batch_size = 128
virus_dataset_name = "corpus_1000_Viruses"
cellular_dataset_name = "corpus_1000_cellular"
model_name = "ML"
max_seq_len = 1000

sys.path.insert(0, '../dlp')
from data_access import PQDataAccess
virus_da = PQDataAccess(f"/home/aac/Alireza/datasets/export_pqt_4_taxseq_new/{virus_dataset_name}", batch_size)
cellular_da = PQDataAccess(f"/home/aac/Alireza/datasets/export_pqt_4_taxseq_new/{cellular_dataset_name}", batch_size)

cuda:1
 WORLD_SIZE=1 , LOCAL_WORLD_SIZE=1,RANK =0,LOCAL_RANK = 0 


In [2]:
import random
index2name_file = "../data/taxonomy_index.json"

# Check if the file exists
if os.path.exists(index2name_file):
    with open(index2name_file, "rb") as f:
        index2name = json.load(f)

tax_vocab_sizes = {
    int(k): len(v) for k,v in index2name.items()
}

level_encoder = {
    int(k): {name: idx + 1 for idx, name in enumerate(v)} for k,v in index2name.items()
}

level_decoder = {
    int(k): {idx + 1: name for idx, name in enumerate(v)} for k,v in index2name.items()
}

for k, v in level_decoder.items():
    level_decoder[k][0] = "NOT DEFINED"


def encode_lineage(lineage_str):
    taxes_str = lineage_str.split(", ")

    encoded = {int(k): 0 for k in index2name.keys()}
    
    for i, tax_str in enumerate(taxes_str):
        encoded[i] = level_encoder[i][tax_str]

    return encoded


from sklearn.preprocessing import LabelEncoder, OneHotEncoder


def one_hot_encode_sequence(seq):
    amino_acids = '-ACDEFGHIKLMNPQRSTVWY'  # 20 standard amino acids
    encoder = OneHotEncoder(categories=[list(amino_acids)], sparse_output=False)

    # Convert sequence into a 2D array (each amino acid as a separate row)
    sequence_array = np.array([aa if aa in amino_acids else '-' for aa in seq] + ['-' for _ in range(max_seq_len - len(seq))]).reshape(-1, 1)

    # Perform one-hot encoding
    one_hot_encoded = encoder.fit_transform(sequence_array)

    # Flatten the one-hot encoded array to a single vector for each sequence
    return one_hot_encoded.flatten()

def mix_data_to_tensor_batch(b_virues, b_cellular, max_seq_len=max_seq_len, partition=0.25):
    split_point = int(len(b_virues) * partition)
    b = b_virues[:split_point] + b_cellular[-len(b_virues) + split_point:]
    random.shuffle(b)  # In-place shuffle
    
    inputs = np.array([one_hot_encode_sequence(e['Sequence']) for e in b])

    tax_ids = [encode_lineage(e['Taxonomic_lineage__ALL_']) for e in b]
    combined_dict = {}
    for d in tax_ids:
        for key, value in d.items():
            combined_dict.setdefault(key, []).append(value)

    encoded = {k: np.array(v) for k, v in combined_dict.items()}
    return {"sequences": inputs, "labels":encoded}

def val_loader():
    val_batches = [virus_da.get_batch() for _ in range(num_val // 2)] + [cellular_da.get_batch() for _ in range(num_val // 2)]

    inputs = np.array([one_hot_encode_sequence(e['Sequence']) for b in val_batches for e in b])

    tax_ids = [encode_lineage(e['Taxonomic_lineage__ALL_']) for b in val_batches for e in b]
    combined_dict = {}
    for d in tax_ids:
        for key, value in d.items():
            combined_dict.setdefault(key, []).append(value)

    encoded = {k: np.array(v) for k, v in combined_dict.items()}
    return {"sequences": inputs, "labels":encoded}

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, \
    classification_report, confusion_matrix
# import xgboost as xgb
#
# # Set up the DMatrix for XGBoost (with GPU)
# dtrain = xgb.DMatrix(X_train, label=y_train)
# dtest = xgb.DMatrix(X_test, label=y_test)
#
# # Set GPU parameters
# params = {
#     'tree_method': 'gpu_hist',  # Use GPU acceleration
#     'device': 'cuda:0',
#     'objective': 'binary:logistic',
#     'eval_metric': 'logloss',
#     'max_depth': 6
# }
#
# # Train the model
# bst = xgb.train(params, dtrain, num_boost_round=100)
#
# # Predict and evaluate
# y_pred = (bst.predict(dtest) > 0.5).astype(int)
# y_proba = bst.predict(dtest)

In [4]:
def get_partition_ratio(epoch, decay_epochs=100000):
    """
    Calculate partition ratio that decreases from 8/16 to 1/16 in steps
    """
    # Calculate how many epochs before each step down
    epochs_per_step = decay_epochs // 7  # 7 steps from 8/16 down to 1/16
    
    # Calculate current step based on epoch
    step = min(epoch // epochs_per_step, 7)  # Max 7 steps down from 8
    
    # Map step to fraction
    fraction = (8 - step) / 16
    
    return fraction

In [ ]:
val_inputs = val_loader()


def evaluate(clf):
    y_test = val_inputs["labels"][0]
    y_pred = clf.predict(val_inputs["sequences"])
    y_proba = clf.predict_proba(val_inputs["sequences"])[:, 1]  # Probabilities for ROC-AUC
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    print(f"Accuracy:{accuracy:.4f}")
    print(f"Precision:{precision:.4f}")
    print(f"Recall:{recall:.4f}")
    print(f"F1-Score:{f1:.4f}")
    print(f"ROC-AUC:{roc_auc:.4f}")
    print("\nConfusion Matrix:\n", conf_matrix)
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    
def get_partition_ratio(epoch, decay_epochs=100000):
    """
    Calculate partition ratio that decreases from 8/16 to 1/16 in steps
    """
    # Calculate how many epochs before each step down
    epochs_per_step = decay_epochs // 7  # 7 steps from 8/16 down to 1/16
    
    # Calculate current step based on epoch
    step = min(epoch // epochs_per_step, 7)  # Max 7 steps down from 8
    
    # Map step to fraction
    fraction = (8 - step) / 16
    
    return fraction


clf = RandomForestClassifier(n_estimators=200, random_state=42)
for epoch in tqdm(range(epochs)):
    inputs = mix_data_to_tensor_batch(virus_da.get_batch(), cellular_da.get_batch(), partition = get_partition_ratio(epoch+1))
    sequences = inputs["sequences"]
    labels = inputs["labels"][0]
    clf.fit(sequences, labels)
    
    if (epoch + 1) % val_epoch == 0:
        print(f"Epoch [{epoch + 1}/{epochs}]")
        evaluate(clf)

  1%|          | 999/100000 [06:25<11:25:35,  2.41it/s]

Epoch [1000/100000]


  1%|          | 1000/100000 [06:27<21:34:08,  1.27it/s]

Accuracy:0.6062
Precision:0.6134
Recall:0.5745
F1-Score:0.5934
ROC-AUC:0.6670

Confusion Matrix:
 [[3677 2723]
 [2317 4083]]

Classification Report:
               precision    recall  f1-score   support

           1       0.61      0.57      0.59      6400
           2       0.60      0.64      0.62      6400

    accuracy                           0.61     12800
   macro avg       0.61      0.61      0.61     12800
weighted avg       0.61      0.61      0.61     12800



  2%|▏         | 1999/100000 [13:01<10:41:52,  2.54it/s]

...

 69%|██████▉   | 68999/100000 [7:23:10<3:10:33,  2.71it/s]

Epoch [69000/100000]


 69%|██████▉   | 69000/100000 [7:23:11<6:04:38,  1.42it/s]

Accuracy:0.5406
Precision:0.5212
Recall:1.0000
F1-Score:0.6852
ROC-AUC:0.6536

Confusion Matrix:
 [[6400    0]
 [5880  520]]

Classification Report:
               precision    recall  f1-score   support

           1       0.52      1.00      0.69      6400
           2       1.00      0.08      0.15      6400

    accuracy                           0.54     12800
   macro avg       0.76      0.54      0.42     12800
weighted avg       0.76      0.54      0.42     12800



 69%|██████▉   | 69256/100000 [7:24:50<3:08:23,  2.72it/s]

In [ ]:
model = ESM1b(tax_vocab_sizes).to(device)
optimizers = model.get_optimizers()


trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f'Trainable parameters: {trainable/ 1e6} M')
print(f'Total parameters: {total/ 1e6} M')
print(model)

# optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
criterion = nn.CrossEntropyLoss()
# Cosine annealing with warm restarts
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizers['0'],
    T_0=10,  # Initial restart interval
    T_mult=2,  # Multiply interval by 2 after each restart
    eta_min=1e-6  # Minimum learning rate
)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

val_dir = f"val_results/{model_name}"
if not os.path.exists(val_dir):
    os.makedirs(val_dir)
    
val_batches = [virus_da.get_batch() for _ in range(num_val // 2)] + [cellular_da.get_batch() for _ in range(num_val // 2)]

input_sequences = [e['Sequence'] for b in val_batches for e in b]
labels_ = [encode_lineage(e['Taxonomic_lineage__ALL_'])  for b in val_batches for e in b]

def evaluate(model):
    model.eval()  # Set model to evaluation mode
    
    df = {
        i : {
            "sequence": [],
            "label": [],
            "pred": [],
            "loss": []
        } for i in tax_vocab_sizes.keys()
    }

    metrics = {
        i : {
            "loss": 0,
            "accuracy": 0,
            "f1 macro": 0,
            "f1 micro": 0
        } for i in tax_vocab_sizes.keys()
    }
    
    # Process each sequence
    for sequence, label in zip(input_sequences, labels_):
        inputs = tokenizer_(
            [sequence],
            return_tensors="pt",
            padding='max_length',
            truncation=True,
            max_length=max_seq_len
        ).to(device)
    
        # Get model output
        with torch.no_grad():
            output = model(inputs['input_ids'], inputs['attention_mask'])

        for k in tax_vocab_sizes.keys():
            pred = output[str(k)].argmax(dim=-1).cpu().item()
            loss = criterion(output[str(k)], torch.tensor([label[k]]).to(device))
            df[k]["sequence"].append(sequence)
            df[k]["label"].append(level_decoder[k][label[k]])
            df[k]["pred"].append(level_decoder[k][pred])
            df[k]["loss"].append(round(loss.cpu().item(), 4))

    for k in tax_vocab_sizes.keys():
        # Convert to DataFrame
        new_df = pd.DataFrame(df[k])
        new_df['is_incorrect'] = new_df['label'] != new_df['pred']
        new_df = new_df.sort_values(['is_incorrect', 'loss'], ascending=[False, False])
        new_df.to_csv(f'val_results/{model_name}/classification_results_{k}.csv', index=False)

        metrics[k]["loss"] = np.array(df[k]["loss"]).mean()
        metrics[k]["accuracy"] = accuracy_score(np.array(df[k]["label"]), np.array(df[k]["pred"]))
        metrics[k]["f1 macro"] = f1_score(np.array(df[k]["label"]), np.array(df[k]["pred"]), average='macro')  # F1-score for multi-label classification
        metrics[k]["f1 micro"] = f1_score(np.array(df[k]["label"]), np.array(df[k]["pred"]), average='micro') 
    
    return metrics

In [ ]:
evaluate(model)

In [ ]:
running_loss = 0
current_lr = lr

for epoch in tqdm(range(epochs)):
    model.train()

    tensor_batch = mix_data_to_tensor_batch(virus_da.get_batch(), cellular_da.get_batch(), partition = get_partition_ratio(epoch+1))
    tensor_batch.gpu(device)
    
    labels = tensor_batch.taxes

    batch_loss = 0
    for index, optimizer in optimizers.items():
        optimizer.zero_grad()
        output = model(tensor_batch.seq_ids['input_ids'], tensor_batch.seq_ids['attention_mask'], index=index)
        loss = criterion(output, labels[int(index)])
        loss.backward()
        optimizer.step()
        batch_loss += loss
    
    running_loss += batch_loss.item()
    
    if (epoch + 1) % val_epoch == 0:
        train_loss = running_loss / val_epoch
        val_metrics = evaluate(model)
        val_losses = {k: v["loss"] for k, v in val_metrics.items()} 
        val_loss = sum([entry['loss'] for entry in val_metrics.values()]) 
        print(f"Epoch [{epoch + 1}/{epochs}]")
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Val Loss: {val_loss:.4f}")
        print(val_losses)
        
        # Create metrics dictionary for saving
        metrics = {
            "train_loss": train_loss,
            "val_loss": val_loss,
            "epoch": epoch + 1,
            "lr": current_lr,
            "partition": get_partition_ratio(epoch+1)
        }
        metrics.update(val_losses)
            
        # Save periodic checkpoint
        checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch}.pt')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizers,
            'scheduler_state_dict': scheduler.state_dict(),
            'metrics': metrics
        }, checkpoint_path)
        
        # Log to wandb
        wandb.log(metrics)

        # Step the scheduler
        scheduler.step(epoch + batch_loss.item())
        current_lr = scheduler.get_last_lr()[0]
        
        # Reset training metrics
        running_loss = 0
wandb.finish()